In [2]:
import ssam
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import anndata as ad

In [3]:
ds = ssam.SSAMDataset("data/processed/ssamdataset-osmFISH.zarr")
analysis = ssam.SSAMAnalysis(ds, ncores=40, verbose=True)

Loaded existing KDE results.
Loaded existing local maxima.
Loaded a precomputed normalized vector field.
Loaded existing cluster labels.
Loaded an existing t-SNE result.
Loaded an existing UMAP result.
Loaded existing cell type maps.
Loaded existing filtered cell type maps.


Loaded existing cell type binned centers and counts.
Loaded existing inferred domains.
Loaded existing watershed segmentations.
Loaded existing transferred cluster labels.


In [4]:
ds.watershed_segments.shape

(1640, 3368)

In [6]:
ds.inferred_domains.shape

(1640, 3368, 1)

In [8]:
np.unique(ds.inferred_domains, return_counts=True)

(array([-1,  0,  1,  2,  3,  4,  5,  6,  7,  8]),
 array([2411227,  296953,  354147,  507688,  256632,  378217, 1040323,
         148679,   60401,   69253]))

In [5]:
import h5py

pixel_per_um = 15.3846 # from BioRxiv paper
um_per_pixel = 1.0 / pixel_per_um

blacklists = ['Cnr1_Hybridization4', 'Plp1_Hybridization4', 'Vtn_Hybridization4',
              'Klk6_Hybridization5', 'Lum_Hybridization9', 'Tbr1_Hybridization11']

with h5py.File("./data/raw/mRNA_coords_raw_counting.hdf5", 'r') as f:
    # Extract gene names (keys)
    gene_names = list(f.keys())
    # Extract x, y coordinates for each gene and store them along with the gene name
    data_list = []
    for gene in gene_names:
        if gene in blacklists:
            continue
        xx, yy = f[gene][:].T
        gene = gene.split("_")[0]
        
        if gene == 'Tmem6':
            gene = 'Tmem2'
        elif gene == 'Kcnip':
            gene = 'Kcnip2'

        for x, y in zip(xx, yy):
            data_list.append([gene, x * um_per_pixel, y * um_per_pixel])

# Create a pandas DataFrame
df = pd.DataFrame(data_list, columns=['gene', 'x', 'y']).set_index('gene')

In [6]:
analysis.compute_cell_by_gene_matrix(df)

Generating spatial mRNA count matrix...


Computing cell-by-gene matrix...


In [12]:
len(ds.cell_by_gene_matrix)

8055

In [7]:
new_element = [0 for i in range(len(ds.genes))]
cell_by_gene_matrix = np.insert(ds.cell_by_gene_matrix, 0, new_element, axis=0)

In [15]:
import scanpy as sc
import spatialdata as sd

cell_type = list()
cell_type.append(0)
for i in range(len(cell_by_gene_matrix)-1):
    cell_type.append(int(np.unique(ds.watershed_celltype_maps[ds.watershed_segments == i]))+1)
    
table_list = []

table1 = sc.AnnData(cell_by_gene_matrix)
table1.var_names = ds.genes
table1.obs["region"] = ["watershed_segments" for i in range(len(cell_by_gene_matrix))]
table1.obs['cell_id'] = np.arange(len(cell_by_gene_matrix))+1
table1.obs['cell_type'] = np.array(cell_type)
table_list.append(table1)

table2 = sc.AnnData(np.unique(ds.inferred_domains))
table2.obs["region"] = ["inferred_domain" for i in range(len(np.unique(ds.inferred_domains)+1))]
table_list.append(table2)

table = ad.concat(
    table_list,
    keys=["watershed_segments","inferred_domain"],
)

expression = sd.models.TableModel.parse(
    adata=table,
    region=["watershed_segments","inferred_domain"],
    region_key='region',
    instance_key="cell_id",
)

celltype_maps = sd.models.Image2DModel.parse(ds.celltype_maps[::-1], dims=('x', 'y', 'c'))
filtered_celltype_maps = sd.models.Image2DModel.parse(ds.filtered_celltype_maps[::-1], dims=('x', 'y', 'c'))
watershed_segments = sd.models.Labels2DModel.parse(ds.watershed_segments[::-1]+1, dims=('x', 'y'))
inferred_domains = sd.models.Labels2DModel.parse(np.squeeze(ds.inferred_domains[::-1]+1), dims=('x', 'y'))
inferred_domains_cells = sd.models.Labels2DModel.parse(np.squeeze(ds.inferred_domains_cells[::-1]+1), dims=('x', 'y'))
inferred_domains_cell = sd.models.Image2DModel.parse(ds.inferred_domains_cells[::-1], dims=('x', 'y', 'c'))
inferred_domain = sd.models.Image2DModel.parse(ds.inferred_domains[::-1], dims=('x', 'y','c'))

sdata = sd.SpatialData(images = {'celltype_maps': celltype_maps, 'filtered_celltype_maps': filtered_celltype_maps, 'inferred_domain_cell': inferred_domains_cell, 'inferred_domain': inferred_domain},labels={'watershed_segments': watershed_segments,'inferred_domains': inferred_domains},table=expression)
sdata.write('data.zarr')

ValueError: X needs to be 2-dimensional, not 1-dimensional.

In [16]:
np.unique(ds.inferred_domains)

array([-1,  0,  1,  2,  3,  4,  5,  6,  7,  8])

In [19]:
ds.inferred_domains[ds.inferred_domains == 1]

array([1, 1, 1, ..., 1, 1, 1])

In [10]:
#원하는 클러스터내 벡터
ds.normalized_vectors[ds.filtered_cluster_labels == 38]

array([[1.82030499e-01, 4.06136632e-01, 1.67381600e-01, 5.03822744e-01,
        1.89701959e-01, 1.88986391e-01, 0.00000000e+00, 1.87325984e-01,
        3.49924743e-01, 1.84833273e-01, 0.00000000e+00, 6.64371066e-04,
        0.00000000e+00, 6.98858723e-02, 1.07306605e-02, 3.07377845e-01,
        3.39207470e-01, 1.79534391e-01, 1.52076423e+00, 4.76837045e-07,
        1.90375745e-01, 2.88917392e-01, 1.93949357e-01, 1.98335513e-01,
        1.92007780e-01, 1.78659052e-01, 1.89247757e-01, 1.75085098e-01,
        9.11178242e-04, 1.88830063e-01, 1.81774661e-01, 1.87677994e-01,
        1.80378184e-01],
       [1.49835706e-01, 8.15818787e-01, 1.55997261e-01, 3.04387420e-01,
        1.51796415e-01, 1.64338097e-01, 0.00000000e+00, 1.42205015e-01,
        1.47825092e-01, 1.45864293e-01, 1.19209275e-07, 1.09584900e-02,
        1.11085773e-02, 5.32008766e-04, 1.08493634e-01, 1.60552204e-01,
        2.63062119e-01, 3.37741911e-01, 1.46274042e+00, 0.00000000e+00,
        2.86482602e-01, 1.67443529e-01,

In [20]:
cell_type = [0]
for i in range(len(cell_by_gene_matrix)-1):
    cell_type.append(int(np.unique(ds.watershed_celltype_maps[ds.watershed_segments == i]))+1)

In [26]:
import numpy as np

main_domain = [0]
secondary_domain = [0]
additional_domains = [0]

unique_domains = [np.unique(ds.inferred_domains[ds.watershed_segments == i]) for i in range(len(cell_by_gene_matrix) - 1)]

for k in unique_domains:
    if len(k) == 1:
        main_domain.append(k[0] + 1)
        secondary_domain.append(k[0]+ 1)
        additional_domains.append(0)
    elif len(k) == 2:
        main_domain.append(k[0] + 1)
        secondary_domain.append(k[1] + 1)
        additional_domains.append(0)
    elif len(k) > 2:
        main_domain.append(k[0] + 1)
        secondary_domain.append(k[1] + 1)
        additional_domains.append(list(k[2:] + 1))
     

In [22]:
import scanpy as sc
import spatialdata as sd
table = sc.AnnData(cell_by_gene_matrix)

table.var_names = ds.genes
table.obs["region"] = ["watershed_segments" for i in range(len(cell_by_gene_matrix))]
table.obs['cell_id'] = np.arange(len(cell_by_gene_matrix))+1
table.obs['cell_type'] = np.array(cell_type)
table.obs['main_domain'] = np.array(main_domain)
table.obs['secondary_domain'] = np.array(secondary_domain)

expression = sd.models.TableModel.parse(
    adata=table,
    region="watershed_segments",
    region_key='region',
    instance_key="cell_id",
)

/home/ng4812/miniconda3/envs/ome/lib/python3.11/site-packages/spatialdata/models/models.py:725: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(adata.obs[region_key]):
/tmp/ipykernel_2822893/439840326.py:12: UserWarning: Converting `region_key: region` to categorical dtype.
  expression = sd.models.TableModel.parse(


In [25]:
import spatialdata as sd
import numpy as np
import scanpy as sc
from spatialdata.transformations import Identity

#transformations={Identity()}
#watershed_segments = sd.models.Labels2DModel.parse(ds.watershed_segments+1, dims=('y', 'x'))
#ValueError: zero-size array to reduction operation fmax which has no identity

celltype_maps = sd.models.Image2DModel.parse(ds.celltype_maps[::-1], dims=('x', 'y', 'c'))
filtered_celltype_maps = sd.models.Image2DModel.parse(ds.filtered_celltype_maps[::-1], dims=('x', 'y', 'c'))
watershed_segments = sd.models.Labels2DModel.parse(ds.watershed_segments[::-1]+1, dims=('x', 'y'))
inferred_domains = sd.models.Labels2DModel.parse(np.squeeze(ds.inferred_domains[::-1]+1), dims=('x', 'y'))
inferred_domains_cells = sd.models.Labels2DModel.parse(np.squeeze(ds.inferred_domains_cells[::-1]+1), dims=('x', 'y'))
inferred_domains_cell = sd.models.Image2DModel.parse(ds.inferred_domains_cells[::-1], dims=('x', 'y', 'c'))
inferred_domain = sd.models.Image2DModel.parse(ds.inferred_domains[::-1], dims=('x', 'y','c'))
#'watershed_celltype_maps': watershed_celltype_maps, 'inferred_domains': inferred_domains,'inferred_domains_cells':inferred_domains_cells

sdata = sd.SpatialData(images = {'celltype_maps': celltype_maps, 'filtered_celltype_maps': filtered_celltype_maps},labels={'watershed_segments': watershed_segments,'inferred_domains': inferred_domains},table=expression)
sdata.write('data.zarr')

INFO     Transposing `data` of type: <class 'dask.array.core.Array'> to ('c', 'y', 'x').                           
INFO     Transposing `data` of type: <class 'dask.array.core.Array'> to ('c', 'y', 'x').                           
INFO     Transposing `data` of type: <class 'dask.array.core.Array'> to ('y', 'x').                                


INFO     Transposing `data` of type: <class 'dask.array.core.Array'> to ('y', 'x').                                
INFO     Transposing `data` of type: <class 'dask.array.core.Array'> to ('y', 'x').                                
INFO     Transposing `data` of type: <class 'dask.array.core.Array'> to ('c', 'y', 'x').                           
INFO     Transposing `data` of type: <class 'dask.array.core.Array'> to ('c', 'y', 'x').                           


no parent found for <ome_zarr.reader.Label object at 0x7f202b011390>: None
no parent found for <ome_zarr.reader.Label object at 0x7f2024195850>: None
/home/ng4812/miniconda3/envs/ome/lib/python3.11/site-packages/spatialdata/_io/format.py:169: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if region_key is not None and not is_categorical_dtype(table.obs[region_key]):


In [24]:
import spatialdata as sd
sdata_read = sd.read_zarr('data.zarr')
sdata_read

PathNotFoundError: nothing found at path ''